# Seven years of Spotify Part One: Extraction

Lately, I've been curious about my personal listening habits. Spotify has an automated tool that delivers personal streaming data from the past year, but I wished to see it all. 

All I had to do was send an email to privacy@spotify.com requesting my extended streaming history. That process, and the description of what data you'll recieve, is [available here](https://support.spotify.com/us/article/data-rights-and-privacy-settings/).

This notebook begins my journey to explore my music taste. Part One encompasses loading in data for everything I've listened to on Spotify since 2015, and using [Spotify's API](https://developer.spotify.com/documentation/web-api/quick-start/), as well as the [well-documented](https://spotipy.readthedocs.io/en/master/#) library, Spotipy, to gather a wide range of metrics surrounding each track, artist, and album. 

In the end, we'll have a dataset ready to be analyzed. 


In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = 'XXXX'
secret = 'XXXX'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [2]:
tracks = pd.concat(
    map(pd.read_json, ['spotify_life/endsong_0.json', 
    'spotify_life/endsong_1.json', 
    'spotify_life/endsong_2.json', 
    'spotify_life/endsong_3.json', 
    'spotify_life/endsong_4.json', 
    'spotify_life/endsong_5.json', 
    'spotify_life/endsong_6.json', 
    'spotify_life/endsong_7.json', 
    'spotify_life/endsong_8.json', 
    'spotify_life/endsong_9.json', 
    'spotify_life/endsong_10.json']), ignore_index=True)
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179546 entries, 0 to 179545
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 179546 non-null  object 
 1   username                           179546 non-null  object 
 2   platform                           179546 non-null  object 
 3   ms_played                          179546 non-null  int64  
 4   conn_country                       179546 non-null  object 
 5   ip_addr_decrypted                  179546 non-null  object 
 6   user_agent_decrypted               179546 non-null  object 
 7   master_metadata_track_name         179044 non-null  object 
 8   master_metadata_album_artist_name  179044 non-null  object 
 9   master_metadata_album_album_name   179044 non-null  object 
 10  spotify_track_uri                  179044 non-null  object 
 11  episode_name                       502 

In [3]:
tracks.describe()

,ms_played,skipped,offline_timestamp
count,1.795460e+05,33833.000000,1.795460e+05
mean,1.125443e+05,0.655514,1.101390e+12
std,1.386337e+05,0.475207,7.362376e+11
min,0.000000e+00,0.000000,0.000000e+00
25%,1.462000e+03,0.000000,0.000000e+00
50%,7.541550e+04,1.000000,1.551370e+12
75%,2.109060e+05,1.000000,1.627057e+12
max,5.813188e+06,1.000000,1.663170e+12


## Extracting data for tracks

The bulk of the work here hinges upon Spotify URI's, or Uniform Resource Indicators. They function as a primary key for every song, track, or album. 

We will use all three throughout this notebook.

In [4]:
unique_track_uris = tracks['spotify_track_uri'].dropna().unique()

def extract_sp_data(unique_uris, sp_method, chunksize):
    """Split a list of unique URIs into n chunks of lists, then perform a spotipy method on each chunk
    
    args: 
    unique_uris (list): unique uris to extract data for
    sp_method (func): an extraction method available in Spotipy
    chunksize (int): number of uris to include in each chunk, to separate the amount of api calls performed. 

    Returns: List of JSON objects for each track
    """

    uris_split = []
    for i in range(0, len(unique_uris), chunksize):
        uris_split.append(unique_uris[i:i+chunksize].tolist())


    item_info = []
    for uris in uris_split:
        item_info.append(sp_method(uris))
    return item_info

track_info = extract_sp_data(unique_track_uris, sp.tracks, 48)

In [5]:
track_info_df = pd.concat(
    map(lambda x: pd.json_normalize(x, record_path=['tracks']), [ls for ls in track_info]), ignore_index=True)
#making sure the artist identifiers also get their own columns, keeping first artist only
track_info_df['artists.name'] = track_info_df['artists'].str[0].str.get('name')
track_info_df['artists.uri'] = track_info_df['artists'].str[0].str.get('uri')
track_info_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify,artists.name,artists.uri
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,272293,False,https://api.spotify.com/v1/tracks/5DiXcVovI0Fc...,5DiXcVovI0FcY2s0icWWUu,False,Mr. Jones,78,...,August And Everything After,1993-01-01,day,11,album,spotify:album:4nKfZbCALT9H9LfedtDwnZ,USIR10000287,https://open.spotify.com/track/5DiXcVovI0FcY2s...,Counting Crows,spotify:artist:0vEsuISMWAKNctLlUAhSZC
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,280520,True,https://api.spotify.com/v1/tracks/17dRP50Rx5Dd...,17dRP50Rx5DdSH9Qy4xR9A,False,American Terrorist (feat. Matthew Santos),34,...,Lupe Fiasco's Food & Liquor,2006-06-27,day,16,album,spotify:album:0TDJRkEr2SrhWTetdkEzED,USAT20620108,https://open.spotify.com/track/17dRP50Rx5DdSH9...,Lupe Fiasco,spotify:artist:01QTIT5P1pFP3QnnFSdsJf
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AM, AO, AR, AT, AU, AZ, BB, BD, B...",1,201760,False,https://api.spotify.com/v1/tracks/4OHnqedhcXex...,4OHnqedhcXexo7f2a5RiZY,False,Walk on the Moon (feat. Lynx),40,...,Walk on the Moon (feat. Lynx),2018-03-06,day,1,album,spotify:album:3rAZDcr1bEh85GiD8UoPPW,TCADM1867865,https://open.spotify.com/track/4OHnqedhcXexo7f...,Luke Mitrani,spotify:artist:6OzAqsb9KgjKjUyoTjZ6UV
3,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,193733,False,https://api.spotify.com/v1/tracks/1oew3nFNY3vM...,1oew3nFNY3vMacJAsvry0S,False,Me And My Broken Heart,79,...,Let The Road,2014-01-01,day,10,album,spotify:album:02ae5i5UAoFrt2peVox9Xd,USUM71401800,https://open.spotify.com/track/1oew3nFNY3vMacJ...,Rixton,spotify:artist:0kkxsdcaWmWU2yWAqclDh4
4,[{'external_urls': {'spotify': 'https://open.s...,[US],1,259346,False,https://api.spotify.com/v1/tracks/5mQrrvxSTuqP...,5mQrrvxSTuqPzLUtaVNtnN,False,Emoticons,35,...,Glitterbug (Deluxe Edition),2015-04-14,day,13,album,spotify:album:0n5sLhq91buJwIW4j3Ji0I,GBAHT1400472,https://open.spotify.com/track/5mQrrvxSTuqPzLU...,The Wombats,spotify:artist:0Ya43ZKWHTKkAbkoJJkwIB


## Extracting Audio Features

In [6]:
audio_features_list = extract_sp_data(unique_track_uris, sp.audio_features, 50)
audio_features_df = pd.concat(
    map(pd.json_normalize, [ls for ls in audio_features_list]), ignore_index=True)
audio_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.578,0.817,0.0,-6.542,1.0,0.0363,0.18300,0.000003,0.285,0.726,141.621,audio_features,5DiXcVovI0FcY2s0icWWUu,spotify:track:5DiXcVovI0FcY2s0icWWUu,https://api.spotify.com/v1/tracks/5DiXcVovI0Fc...,https://api.spotify.com/v1/audio-analysis/5DiX...,272293.0,4.0
1,0.614,0.959,1.0,-3.514,1.0,0.0930,0.01680,0.000000,0.312,0.484,83.264,audio_features,17dRP50Rx5DdSH9Qy4xR9A,spotify:track:17dRP50Rx5DdSH9Qy4xR9A,https://api.spotify.com/v1/tracks/17dRP50Rx5Dd...,https://api.spotify.com/v1/audio-analysis/17dR...,280520.0,4.0
2,0.810,0.637,11.0,-8.029,1.0,0.0424,0.01660,0.000001,0.112,0.622,106.938,audio_features,4OHnqedhcXexo7f2a5RiZY,spotify:track:4OHnqedhcXexo7f2a5RiZY,https://api.spotify.com/v1/tracks/4OHnqedhcXex...,https://api.spotify.com/v1/audio-analysis/4OHn...,201760.0,4.0
3,0.545,0.783,5.0,-4.261,0.0,0.0345,0.00489,0.000000,0.132,0.551,174.084,audio_features,1oew3nFNY3vMacJAsvry0S,spotify:track:1oew3nFNY3vMacJAsvry0S,https://api.spotify.com/v1/tracks/1oew3nFNY3vM...,https://api.spotify.com/v1/audio-analysis/1oew...,193733.0,4.0
4,0.610,0.708,11.0,-4.300,0.0,0.0606,0.00950,0.000031,0.116,0.508,101.003,audio_features,5mQrrvxSTuqPzLUtaVNtnN,spotify:track:5mQrrvxSTuqPzLUtaVNtnN,https://api.spotify.com/v1/tracks/5mQrrvxSTuqP...,https://api.spotify.com/v1/audio-analysis/5mQr...,259347.0,4.0


## Extracting Artist info

In [7]:
unique_artist_uris = track_info_df['artists.uri'].unique()
artist_info = extract_sp_data(unique_artist_uris, sp.artists, 37)
artist_info_df = pd.concat(
    map(lambda x: pd.json_normalize(x, record_path=['artists']), [ls for ls in artist_info]), ignore_index=True)
artist_info_df.head()

,genres,href,id,images,name,popularity,type,uri,external_urls.spotify,followers.href,followers.total
0,"[alternative rock, neo mellow, pop rock, rock]",https://api.spotify.com/v1/artists/0vEsuISMWAK...,0vEsuISMWAKNctLlUAhSZC,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Counting Crows,68,artist,spotify:artist:0vEsuISMWAKNctLlUAhSZC,https://open.spotify.com/artist/0vEsuISMWAKNct...,None,1586363
1,"[alternative hip hop, chicago rap, conscious h...",https://api.spotify.com/v1/artists/01QTIT5P1pF...,01QTIT5P1pFP3QnnFSdsJf,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Lupe Fiasco,66,artist,spotify:artist:01QTIT5P1pFP3QnnFSdsJf,https://open.spotify.com/artist/01QTIT5P1pFP3Q...,None,1817005
2,[nyc pop],https://api.spotify.com/v1/artists/6OzAqsb9Kgj...,6OzAqsb9KgjKjUyoTjZ6UV,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Luke Mitrani,41,artist,spotify:artist:6OzAqsb9KgjKjUyoTjZ6UV,https://open.spotify.com/artist/6OzAqsb9KgjKjU...,None,8793
3,"[dance pop, pop, pop rock, post-teen pop]",https://api.spotify.com/v1/artists/0kkxsdcaWmW...,0kkxsdcaWmWU2yWAqclDh4,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Rixton,62,artist,spotify:artist:0kkxsdcaWmWU2yWAqclDh4,https://open.spotify.com/artist/0kkxsdcaWmWU2y...,None,739070
4,"[liverpool indie, modern rock, rock]",https://api.spotify.com/v1/artists/0Ya43ZKWHTK...,0Ya43ZKWHTKkAbkoJJkwIB,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Wombats,66,artist,spotify:artist:0Ya43ZKWHTKkAbkoJJkwIB,https://open.spotify.com/artist/0Ya43ZKWHTKkAb...,None,1026603


## Extracting album info

In [8]:
unique_album_uris = track_info_df['album.uri'].unique()
album_info = extract_sp_data(unique_album_uris, sp.albums, 20)
album_info_df = pd.concat(
    map(lambda x: pd.json_normalize(x, record_path=['albums']), [ls for ls in album_info]), ignore_index=True)
album_info_df.head()

,album_type,artists,available_markets,copyrights,genres,href,id,images,label,name,...,external_ids.upc,external_urls.spotify,tracks.href,tracks.items,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total,external_ids.amgid
0,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...","[{'text': '© 1993 DGC Records', 'type': 'C'}, ...",[],https://api.spotify.com/v1/albums/4nKfZbCALT9H...,4nKfZbCALT9H9LfedtDwnZ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",DGC,August And Everything After,...,00720642452820,https://open.spotify.com/album/4nKfZbCALT9H9Lf...,https://api.spotify.com/v1/albums/4nKfZbCALT9H...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,11,NaN
1,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",[{'text': '© 2006 Atlantic Recording Corporati...,[],https://api.spotify.com/v1/albums/0TDJRkEr2Srh...,0TDJRkEr2SrhWTetdkEzED,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Atlantic Records,Lupe Fiasco's Food & Liquor,...,075678395963,https://open.spotify.com/album/0TDJRkEr2SrhWTe...,https://api.spotify.com/v1/albums/0TDJRkEr2Srh...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,16,NaN
2,single,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AM, AO, AR, AT, AU, AZ, BB, BD, B...","[{'text': '2018 Frendly Presents', 'type': 'C'...",[],https://api.spotify.com/v1/albums/3rAZDcr1bEh8...,3rAZDcr1bEh85GiD8UoPPW,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Frendly Presents,Walk on the Moon (feat. Lynx),...,859725358329,https://open.spotify.com/album/3rAZDcr1bEh85Gi...,https://api.spotify.com/v1/albums/3rAZDcr1bEh8...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,1,NaN
3,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",[{'text': '© 2014 School Boy/Giant Little Man/...,[],https://api.spotify.com/v1/albums/02ae5i5UAoFr...,02ae5i5UAoFrt2peVox9Xd,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Silent Records/Giant Little Man,Let The Road,...,00602537979912,https://open.spotify.com/album/02ae5i5UAoFrt2p...,https://api.spotify.com/v1/albums/02ae5i5UAoFr...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,10,NaN
4,album,[{'external_urls': {'spotify': 'https://open.s...,[US],[{'text': '© 2015 Bright Antenna Records under...,[],https://api.spotify.com/v1/albums/0n5sLhq91buJ...,0n5sLhq91buJwIW4j3Ji0I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Bright Antenna,Glitterbug (Deluxe Edition),...,856908004295,https://open.spotify.com/album/0n5sLhq91buJwIW...,https://api.spotify.com/v1/albums/0n5sLhq91buJ...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,13,NaN


## Condensing into one dataframe

Of the three new dataframes, we'll want these columns to be joined with the first dataframe of lifetime listening data. 

- track_info_df: `uri`, `album.uri`, `artists.uri`, `popularity`, `duration_ms`, `album.album_type`, `album.release_date`
- artist_info_df: `genres`, `popularity`, `followers.total`
- album_info_df: `popularity`, `label`, `tracks.total`
- audio_features_df: `uri`, `danceability`, `energy`, `key`, `loudness`, `mode`, `speechiness`, `acousticness`, `instrumentalness`, `liveness`, `valence`, `tempo`, `time_signature`

Before we can work with the lifetime set, we'll want everything to get linked to the track_info_df, which hosts each item's unique ID (the URI)

In [9]:
track_info_cols = ['uri', 'album.uri', 'artists.uri', 'popularity', 'duration_ms', 'album.album_type', 'album.release_date']
artist_info_cols = ['uri', 'genres', 'popularity', 'followers.total']
album_info_cols = ['uri', 'popularity', 'label', 'tracks.total']
audio_features_cols = ['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

track_info_reduced = track_info_df[track_info_cols].rename(columns={'uri':'track_uri', 'popularity':'track_popularity'})
artist_info_reduced = artist_info_df[artist_info_cols].rename(columns={'uri':'artists.uri', 'popularity':'artist_popularity'})
album_info_reduced = album_info_df[album_info_cols].rename(columns={'uri':'album.uri', 'popularity':'album_popularity'})
audio_features_reduced = audio_features_df[audio_features_cols].rename(columns={'uri':'track_uri'})

track_info_combined = track_info_reduced.merge(artist_info_reduced, on='artists.uri')
track_info_combined = track_info_combined.merge(album_info_reduced, on='album.uri')
track_info_combined = track_info_combined.merge(audio_features_reduced, on='track_uri')
track_info_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19305 entries, 0 to 19304
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   track_uri           19305 non-null  object 
 1   album.uri           19305 non-null  object 
 2   artists.uri         19305 non-null  object 
 3   track_popularity    19305 non-null  int64  
 4   duration_ms         19305 non-null  int64  
 5   album.album_type    19305 non-null  object 
 6   album.release_date  19305 non-null  object 
 7   genres              19305 non-null  object 
 8   artist_popularity   19305 non-null  int64  
 9   followers.total     19305 non-null  int64  
 10  album_popularity    19305 non-null  int64  
 11  label               19296 non-null  object 
 12  tracks.total        19305 non-null  int64  
 13  danceability        19305 non-null  float64
 14  energy              19305 non-null  float64
 15  key                 19305 non-null  float64
 16  loud

## Adding new features back into the dataframe of lifetime listening

In [10]:
full_tracks = tracks.merge(track_info_combined, left_on='spotify_track_uri', right_on='track_uri', how='left')
#dropping PII
full_tracks.drop(columns=['username', 'platform', 'conn_country', 'ip_addr_decrypted', 'user_agent_decrypted'], axis=1, inplace=True)
#dropping podcast cols
full_tracks.drop(columns=['episode_name', 'episode_show_name', 'spotify_episode_uri'], axis=1, inplace=True)
full_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179565 entries, 0 to 179564
Data columns (total 38 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 179565 non-null  object 
 1   ms_played                          179565 non-null  int64  
 2   master_metadata_track_name         179063 non-null  object 
 3   master_metadata_album_artist_name  179063 non-null  object 
 4   master_metadata_album_album_name   179063 non-null  object 
 5   spotify_track_uri                  179063 non-null  object 
 6   reason_start                       179565 non-null  object 
 7   reason_end                         179565 non-null  object 
 8   shuffle                            179565 non-null  bool   
 9   skipped                            33837 non-null   float64
 10  offline                            179565 non-null  bool   
 11  offline_timestamp                  1795

In [11]:
full_tracks.to_csv('lifetime_spotify_tracks.csv', index=False)